In [5]:
import geopandas as gpd
import pandas as pd
import numpy as np
from geopandas import GeoDataFrame
from shapely.geometry import Point, LineString
import matplotlib
from openpyxl import load_workbook
from datetime import datetime

In [6]:
gen_data = pd.read_csv('Monthly generation states 2018-09-23.csv')
Texas_gen_data = gen_data[gen_data['state']=='TX'].reset_index(drop=True)
Texas_gen_data_2017 = Texas_gen_data[Texas_gen_data['year']==2017].drop(['year', 'state', 'total fuel (mmbtu)', 'elec fuel (mmbtu)', 'elec fuel co2 (kg)', 'datetime', 'quarter'], axis=1).reset_index(drop=True)

In [7]:
fuel_cat = list(Texas_gen_data_2017['fuel category'].unique())

In [8]:
writer = pd.ExcelWriter('Fuel_categories_gen.xlsx', engine = 'openpyxl')

Total= []

for fuel in fuel_cat:
    df = Texas_gen_data_2017[Texas_gen_data_2017['fuel category']==fuel].drop('fuel category', axis=1).reset_index(drop=True)
    Total.append(df)
    df.to_excel(writer, sheet_name = str(fuel), index=False)

total_df = np.zeros((12,2))

for df in Total:
     total_df += df.iloc[:,1:].as_matrix()

total_df = pd.DataFrame(data = total_df, columns = ['generation (mwh)', 'co2 emissions (kg)'])
total_df['Month'] = pd.Series(range(1,13))
total_df.to_excel(writer, sheet_name = 'Total', index=False)

writer.save()
writer.close()

In [9]:
Load = pd.read_excel('Data/Texas_Load_2017/native_Load_2017.xlsx')

In [10]:
def convert_time(x):
    t = x.split(' ')
    t2 = t[1].split(':')
    t3 = int(t2[0])-1
    t = t[0] + ' ' + str(t3) + ':' + t2[1]
    return datetime.strptime(t, '%m/%d/%Y %H:%M')

In [11]:
convert_time(Load['Hour Ending'][22])

datetime.datetime(2017, 1, 1, 22, 0)

In [12]:
Load['Time'] = Load['Hour Ending'].apply(convert_time)

In [13]:
Load.drop('Hour Ending', axis=1, inplace = True)
Load.set_index("Time", drop=True, inplace=True)
Load = Load.resample('M').sum().reset_index()

In [14]:
Load['Month'] = Load['Time'].dt.month

In [15]:
Load.drop('Time', axis = 1, inplace=True)

In [16]:
book = load_workbook('Fuel_categories_gen.xlsx')
writer = pd.ExcelWriter('Fuel_categories_gen.xlsx', engine = 'openpyxl')
writer.book = book
# writer = pd.ExcelWriter('Fuel_categories_gen.xlsx', engine = 'openpyxl')
Load.to_excel(writer, sheet_name = 'Load', index=False)
writer.save()
writer.close()